<a href="https://colab.research.google.com/github/YellaCRE/DACON/blob/main/2022.05.31_Dacon_%EC%A4%91%EA%B3%A0%EC%B0%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#필요 라이브러리설치

In [ ]:
!pip install pycaret[full]

In [ ]:
!pip install -U pandas-profiling

In [ ]:
!pip install MarkupSafe==2.0.1

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#데이터 불러오기

In [ ]:
import os
import pandas as pd
import pandas_profiling
pd.set_option('mode.chained_assignment',  None) #수정오류 무시
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
#csv 형식으로 된 데이터 파일을 읽어옵니다. 
train = pd.read_csv('/content/drive/MyDrive/DACON/2022.05.31 중고차가격/data/train.csv')
test = pd.read_csv('/content/drive/MyDrive/DACON/2022.05.31 중고차가격/data/test.csv')
submission = pd.read_csv('/content/drive/MyDrive/DACON/2022.05.31 중고차가격/data/sample_submission.csv')

#결측치 확인

In [ ]:
def check_missing_col(dataframe):
    missing_col = []
    counted_missing_col = 0
    for i, col in enumerate(dataframe.columns):
        missing_values = sum(dataframe[col].isna())
        is_missing = True if missing_values >= 1 else False
        if is_missing:
            counted_missing_col += 1
            print(f'결측치가 있는 컬럼은: {col}입니다')
            print(f'해당 컬럼에 총 {missing_values}개의 결측치가 존재합니다.')
            missing_col.append([col, dataframe[col].dtype])
    if counted_missing_col == 0:
        print('결측치가 존재하지 않습니다')
    return missing_col

missing_col = check_missing_col(train)

결측치가 존재하지 않습니다


In [ ]:
# 결측치를 처리하는 함수를 작성합니다.
def handle_na(data, missing_col):
    temp = data.copy()
    for col, dtype in missing_col:
        if dtype == 'O':
            # 범주형 feature가 결측치인 경우 해당 행들을 삭제해 주었습니다.
            temp = temp.dropna(subset=[col])
    return temp

train = handle_na(train, missing_col)

# 결측치 처리가 잘 되었는지 확인해 줍니다.
missing_col = check_missing_col(train) 

결측치가 존재하지 않습니다


#EDA

In [ ]:
df = pd.concat([train, test], axis=0)
df.head(5)

,id,title,odometer,location,isimported,engine,transmission,fuel,paint,year,target
0,0,Toyota RAV 4,18277,Lagos,Foreign Used,4-cylinder(I4),automatic,petrol,Red,2016,13665000.0
1,1,Toyota Land Cruiser,10,Lagos,New,4-cylinder(I4),automatic,petrol,Black,2019,33015000.0
2,2,Land Rover Range Rover Evoque,83091,Lagos,Foreign Used,6-cylinder(V6),automatic,petrol,Red,2012,9915000.0
3,3,Lexus ES 350,91524,Lagos,Foreign Used,4-cylinder(I4),automatic,petrol,Gray,2007,3815000.0
4,4,Toyota Venza,94177,Lagos,Foreign Used,6-cylinder(V6),automatic,petrol,Red,2010,7385000.0


In [ ]:
#df.profile_report()

#전처리

In [ ]:
#브랜드랑 차종으로 나누기
temp = df.title.str.split(' ')
brand = []
carname = []

for b in temp:
  if b[0] == 'Land':
    brand.append(b[0]+' '+b[1])
    carname.append(" ".join(b[2:]))
  else:
    brand.append(b[0])
    carname.append(" ".join(b[1:]))

df.insert(1,'brand',brand,True)
df['title']=carname

In [ ]:
#색 이종분류
color = []
famous=['BLACK','GRAY','WHITE','GREY','SLIVER','MIDNIGHT BLACK METAL','SILVER ','BLACK ','DARK GRAY','SLIVER', 
        'WHITE ORCHILD PEARL','GREY ','WHITE ','MAGNETIC GRAY', 'GRAY ', 'DARK SILVER ','BLACK.', 'SILVER/GREY',
        'GERY','BLACK AND SILVER', 'LIGHT SILVER ', 'BLACK SAND PEARL','OFF WHITE', 'ASH','SUPER WHITE', ' BLACK',
        'ASH AND BLACK','MAGNETIC GRAY METALLIC','OFF WHITE L', 'SLIVER ','LIGHT GREY', 'BLAC','GREY AND BLACK',
        'CLASSIC SILVER MET(1F7)',]
        
for p in df['paint'].str.upper():
  if p in famous:
    color.append('famous')
  else:
    color.append('unique')

df['paint'] = color

Preprocessing

In [ ]:
train = df[df['target'].notnull()]
test = df[df['target'].isnull()].drop('target', axis=1)

Scaling

In [ ]:
con_feature = ['odometer','year']
cat_feature = list(set(list(train.columns)) - set(['id','target']) - set(con_feature))
print(con_feature, cat_feature, sep='\n')

['odometer', 'year']
['brand', 'location', 'isimported', 'paint', 'engine', 'transmission', 'fuel', 'title']


In [ ]:
from sklearn.compose import ColumnTransformer 
from sklearn.preprocessing import RobustScaler

# 연속형 변수 Scaling
train_feature, test_feature = train.loc[:, con_feature], test.loc[:, con_feature]
scaler = RobustScaler().fit(train_feature)
train.loc[:, con_feature] = scaler.transform(train_feature)
test.loc[:, con_feature] = scaler.transform(test_feature)

#모델링 파이캐럿

In [ ]:
import optuna
from pycaret.regression import * #regression,classification

/usr/local/lib/python3.7/dist-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


In [ ]:
s = setup(
    data = train,
    target = 'target',
    numeric_features = con_feature,
    categorical_features = cat_feature,
    train_size = 0.85,
    fold=5,
    rare_level_threshold = 0.05,
    ignore_features = ['id'],
    )

,Description,Value
0,session_id,3790
1,Target,target
2,Original Data,"(1015, 12)"
3,Missing Values,False
4,Numeric Features,2
5,Categorical Features,8
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(862, 239)"


In [ ]:
n_models = compare_models(sort = 'RMSE', n_select=5) # 성능이 가장 우수한 상위 5개 모델

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
catboost,CatBoost Regressor,2.207206e+06,5.707881e+13,6.478472e+06,7.144000e-01,0.4174,3.180000e-01,1.972
gbr,Gradient Boosting Regressor,2.612088e+06,6.080795e+13,6.963215e+06,6.776000e-01,0.4592,4.105000e-01,0.266
xgboost,Extreme Gradient Boosting,2.358623e+06,6.075912e+13,7.032938e+06,6.680000e-01,0.4333,3.076000e-01,1.070
rf,Random Forest Regressor,2.321442e+06,6.234636e+13,7.057228e+06,6.688000e-01,0.3949,2.913000e-01,0.984
et,Extra Trees Regressor,2.338448e+06,6.887499e+13,7.725119e+06,5.988000e-01,0.4025,2.924000e-01,1.148
dt,Decision Tree Regressor,2.726936e+06,7.231591e+13,7.981515e+06,5.646000e-01,0.4765,3.298000e-01,0.032
knn,K Neighbors Regressor,3.289161e+06,7.913466e+13,8.343865e+06,5.317000e-01,0.5621,5.715000e-01,0.086
lightgbm,Light Gradient Boosting Machine,3.672403e+06,8.136882e+13,8.587848e+06,4.931000e-01,0.6515,5.591000e-01,0.206
ada,AdaBoost Regressor,7.010969e+06,1.152976e+14,1.058365e+07,2.011000e-01,1.1004,2.134500e+00,0.156
br,Bayesian Ridge,6.602181e+06,1.521765e+14,1.205854e+07,-4.200000e-03,1.1074,1.846000e+00,0.058


catboost

In [ ]:
catboost_tuned = tune_model(
    create_model('catboost'),
    n_iter = 10,
    optimize = 'RMSE',
    )

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,3.087524e+06,1.714402e+14,1.309352e+07,0.3711,0.5726,0.3199
1,1.574846e+06,8.331749e+12,2.886477e+06,0.8967,0.3905,0.3361
2,2.208145e+06,3.014784e+13,5.490705e+06,0.7591,0.3921,0.3382
3,2.429217e+06,6.774461e+13,8.230712e+06,0.6039,0.4622,0.3288
4,1.695684e+06,1.106259e+13,3.326048e+06,0.8980,0.3238,0.2572
Mean,2.199083e+06,5.774540e+13,6.605492e+06,0.7058,0.4282,0.3160
Std,5.450602e+05,6.067802e+13,3.756711e+06,0.1993,0.0844,0.0301


In [ ]:
#데이터 저장
catboost_pred = predict_model(catboost_tuned, test)
submission['target'] = catboost_pred['Label']
submission.to_csv('catboost_submission.csv', index=False)

Voting Classifier

In [ ]:
tuned_n_model = [tune_model(i, 
                            n_iter = 10, 
                            optimize='RMSE', 
                            search_library='optuna', 
                            choose_better=True) for i in n_models]

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,3.347258e+06,1.866058e+14,1.366037e+07,0.3154,0.4936,0.3312
1,2.095487e+06,1.944646e+13,4.409814e+06,0.7589,0.4085,0.3564
2,2.396575e+06,4.375872e+13,6.615037e+06,0.6503,0.3837,0.3431
3,2.976120e+06,6.379581e+13,7.987228e+06,0.6270,0.4941,0.4165
4,2.141764e+06,2.037019e+13,4.513334e+06,0.8122,0.3585,0.2919
Mean,2.591441e+06,6.679540e+13,7.437158e+06,0.6328,0.4277,0.3478
Std,4.909971e+05,6.211513e+13,3.388818e+06,0.1727,0.0563,0.0405


In [ ]:
voting = blend_models(tuned_n_model,choose_better=True)
predict_model(voting)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,3.133632e+06,1.736635e+14,1.317815e+07,0.3629,0.4596,0.3002
1,1.806994e+06,1.104454e+13,3.323333e+06,0.8631,0.3851,0.3417
2,2.114423e+06,2.974069e+13,5.453502e+06,0.7624,0.3423,0.3025
3,2.471029e+06,6.158737e+13,7.847762e+06,0.6399,0.4737,0.3915
4,1.702717e+06,1.190353e+13,3.450149e+06,0.8903,0.3160,0.2520
Mean,2.245759e+06,5.758793e+13,6.650578e+06,0.7037,0.3953,0.3176
Std,5.184061e+05,6.085780e+13,3.654824e+06,0.1918,0.0624,0.0466


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,CatBoost Regressor,2.690892e+06,5.407077e+13,7.353283e+06,0.6643,0.4203,0.3186


,odometer,year,brand_Acura,brand_Audi,brand_BMW,brand_Bentley,brand_Chevrolet,brand_DAF,brand_Dodge,brand_Ford,...,engine_4-cylinder(I4),engine_5-cylinder(I5),engine_6-cylinder(I6),engine_6-cylinder(V6),engine_8-cylinder(V8),transmission_manual,fuel_diesel,paint_unique,target,Label
0,0.209278,0.125,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,6840000.0,4.250375e+06
1,-0.216318,0.750,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15015000.0,1.024141e+07
2,2.068680,-0.875,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,765000.0,2.079158e+05
3,-0.127967,0.750,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,47015000.0,3.494798e+07
4,-0.036288,0.125,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3205000.0,3.690169e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,-0.393182,0.625,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,11515000.0,1.413569e+07
149,-0.265636,0.750,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,18015000.0,1.877940e+07
150,-0.021098,0.375,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3215000.0,4.334532e+06
151,-0.271350,0.125,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3215000.0,3.272707e+06


In [ ]:
#데이터 저장
voting_pred = predict_model(voting, test)
submission['target'] = voting_pred['Label']
submission.to_csv('voting_submission.csv', index=False)